In [13]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
from scipy.fft import dctn, idctn

Variables Definition

In [14]:
img = img_array = img_label = original_img = None
label_dimensions = label_img = None
label_F = entry_F = lock_button_F = None
label_d = entry_d = lock_button_d = None
div1 = div2 = None
root = tk.Tk()

In [15]:
def create_input_component(title, button_name, function):
    label = tk.Label(root, text=title)
    entry = tk.Entry(root)
    lock_button = tk.Button(root, text=button_name, command=function)

    return label, entry, lock_button


def upload_image():
    global img, img_array, original_img
    file_path = filedialog.askopenfilename(filetypes=[("BMP files", "*.bmp")])
    if file_path:
        img = Image.open(file_path)
        original_img = img.copy()
        img.convert('L')
        img_array = np.array(img)

        if img_array.ndim > 2:
            img_array = img_array[:, :, 0]
        
        img.thumbnail((300, 300))
        img_tk = ImageTk.PhotoImage(img)
        img_label.config(image=img_tk)
        img_label.image = img_tk

    label_dimensions.config(text=f"Dimensions: {img_array.shape[1]}x{img_array.shape[0]}")
    label_dimensions.pack(pady=10)
    div1.pack(padx=10, pady=10, fill="both", expand=True)
    label_F.pack()
    entry_F.pack(pady=10)
    lock_button_F.pack(pady=5)

def lock_F():
    try:
        F = int(entry_F.get())
        if F <= 0 and F > img_array.shape[0] or F > img_array.shape[1]:
            raise ValueError
    except ValueError:
        messagebox.showerror("Warning", "F value is wrong. Retry!")
        return

    # Disabilita l'input per F e il pulsante di blocco
    entry_F.config(state='disabled')
    lock_button_F.config(state='disabled')

    # Aggiorna e mostra l'etichetta e l'entry per d
    div2.pack(padx=10, pady=10, fill="both", expand=True)
    # Aggiorna e mostra l'etichetta e l'entry per d
    label_d.config(text=f"Insert value of d (0 ≤ d ≤ {2*F - 2}):")
    label_d.pack()
    entry_d.pack(pady=10)
    lock_button_d.pack(pady=5)


def resize_image_keep_ratio(image, target_width=None, target_height=None):
    original_width, original_height = image.size

    # Calcola il rapporto di aspetto
    aspect_ratio = original_width / original_height

    # Calcola le nuove dimensioni mantenendo il rapporto di aspetto
    if target_width and target_height:
        new_width = target_width
        new_height = int(new_width / aspect_ratio)
        if new_height > target_height:
            new_height = target_height
            new_width = int(new_height * aspect_ratio)
    elif target_width:
        new_width = target_width
        new_height = int(new_width / aspect_ratio)
    elif target_height:
        new_height = target_height
        new_width = int(new_height * aspect_ratio)
    else:
        new_width = original_width
        new_height = original_height

    # Ridimensiona l'immagine
    return image.resize((new_width, new_height))


def create_widgets(title, parent, image, width, height):
    new_window = tk.Toplevel(parent)
    new_window.geometry(f"{width}x{height}")
    new_window.maxsize(width, height)
    new_window.title(title)

    resize_image = resize_image_keep_ratio(image, width, height)
    image_tk = ImageTk.PhotoImage(resize_image)
    
    img_label = tk.Label(new_window, image=image_tk)
    img_label.image = image_tk
    img_label.pack()

def apply_dct_operations():

    try:
        F = int(entry_F.get())
        d = int(entry_d.get())
        if d < 0 or d > (2 * F - 2):
            raise ValueError
    except ValueError:
        messagebox.showerror("Warning", f"d must be an integer between 0 and {2*F - 2}")
        return

    height, width,  = img_array.shape

    # Calculate the maximum height and width that are divisible by F
    height = (height // F) * F
    width = (width // F) * F

    # Crop the image to the calculated dimensions
    image_to_compress = img_array[:height, :width]

    block_height = height // F
    block_width = width // F

    img_dct_array = np.zeros((height, width))
    
    for i in range(block_height):
        for j in range(block_width):
            block_start_i = i * F
            block_end_i = block_start_i + F
            block_start_j = j * F
            block_end_j = block_start_j + F
            block = image_to_compress[block_start_i:block_end_i, block_start_j:block_end_j]
            
            c = dctn(block, norm='ortho')

            for k in range(F):
                for l in range(F):
                    if k + l >= d:
                        c[k, l] = 0

            img_dct_array[block_start_i:block_end_i, block_start_j:block_end_j] = c

            block_idct = idctn(c, norm='ortho')

            # Round and clip block_idct
            block_idct = np.round(block_idct).astype(int)
            block_idct = np.clip(block_idct, 0, 255)

            # Update img_dct_array
            img_dct_array[block_start_i:block_end_i, block_start_j:block_end_j] = block_idct

    modified_img = Image.fromarray(img_dct_array.astype(np.uint8))
    
    create_widgets("Modified Image", root, modified_img, 1200, 800)
    create_widgets("Original Image", root, original_img, 1200, 800)
    

def create_div(parent):
    div = tk.Frame(parent, bg="lightgray", bd=2, relief="ridge")
    return div

In [16]:
# Crea la finestra Tkinter
root.title("DCT2 Image Compression")
root.attributes("-topmost", True)
root.minsize(500, 300)

# Crea e posiziona i widget
upload_button = tk.Button(root, text="Upload Image", command=upload_image)
upload_button.pack(pady=10)

img_label = tk.Label(root)
img_label.pack(pady=10)

div3 = create_div(root)
label_dimensions= tk.Label(root)

div1 = create_div(root)
label_F, entry_F, lock_button_F = create_input_component("Insert value of F (block dimension):", "Confirm", lock_F)


div2 = create_div(root)
label_d, entry_d, lock_button_d = create_input_component("", "Run DCT2", apply_dct_operations)

root.mainloop()